# Collaborative filtering con surprise

Fuente: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
from scipy import spatial
import surprise
import surprise.model_selection

from sklearn.metrics.pairwise import cosine_similarity

from tqdm.notebook import tqdm

import time

In [3]:
BASE_PATH = "dataset"
movies = pd.read_csv(f"{BASE_PATH}/movies_metadata.csv", low_memory=False, dtype={'id':str, 'original_title':str})
user_ratings = pd.read_csv(f"{BASE_PATH}/ratings.csv", dtype={'userId': int, 'movieId': int, 'rating': float,'timestamp': int})
id_links = pd.read_csv(f"{BASE_PATH}/links.csv", dtype={'movieId': str, 'tmdbId': str})

In [4]:
# Remuevo los IDs problemáticos y casteo los movie ids a ints para que sea todo más rápido
movies = movies[
    (movies["id"] != "1997-08-20") &
    (movies["id"] != "2012-09-29") &
    (movies["id"] != "2014-01-01")
]
movies = movies.astype({"id": int})
id_links = id_links.dropna().astype({"tmdbId": int, "movieId": int}) # dropna porque tmdbId es NaN a veces

In [5]:
# Sacamos las películas duplicadas, algunas como id 69234 aparecen dos veces
len_before = len(movies)
movies = movies.drop_duplicates(subset=["id"]).reset_index()
print(f"before: {len_before}, after: {len(movies)}, diff: {len_before - len(movies)}")

before: 45463, after: 45433, diff: 30


In [6]:
# Extend user ratings with additional information, movie ids
user_ratings = pd.merge(user_ratings, id_links[["movieId", "tmdbId"]], left_on='movieId', right_on='movieId', how='left')
# Dropna porque hay algunas columnas de id_links que no tienen tmdbId,
# entonces tampoco los vamos a tener en user_ratings.
user_ratings.dropna(axis=0, subset=["tmdbId"], inplace=True)

In [7]:
def get_movie_name(movie_tmdb_id: str) -> str:
    return movies[movies["id"] == movie_tmdb_id]["original_title"][0]

def get_movie_id(movie_name: str) -> str:
    return movies[movies["original_title"] == movie_name]["id"].iloc[0]

In [8]:
def decorate_with_titles(df: pd.DataFrame):
    df_with_titles = pd.merge(df, movies[["id", "original_title"]], left_on="tmdbId", right_on="id", how="left")
    return df_with_titles.drop('id', axis=1) # 1 = columns
    

In [9]:
# movie names
#user_ratings_titles = decorate_with_titles(user_ratings_ids)

In [10]:
#user_ratings_small = user_ratings_titles[:len(user_ratings_titles) //128]
#user_ratings_small = user_ratings_titles.sample(len(user_ratings_titles) // 2**4)
user_ratings_small = user_ratings
print(f"{len(user_ratings_small)} of {len(user_ratings)}")

26010786 of 26010786


In [11]:
def timestep(start: float, name: str) -> float:
    now = time.time()
    print(f"{name}: {now - start}")
    return now

In [12]:
def predict_ratings_for_movies(model, user_id: int, train) -> pd.DataFrame:
    rows = []

    movie_ids = movies["id"]
    movies_rated_by_user = train[train['userId'] == user_id]["tmdbId"]
    movie_ids = remove_rated_movies(movie_ids, movies_rated_by_user)

    for _, movie_id in movie_ids.items():
        pred = model.predict(user_id, movie_id)
        rows.append({"tmdbId": movie_id, "est_rating": pred.est})

    df = pd.DataFrame.from_records(rows)
    top_10_movies = df.sort_values(by="est_rating", ascending=False).head(10)
    
    # Comentado porque es muy lento y no hace falta
    #top_10_movies = decorate_with_titles(top_10_movies)
    
    return top_10_movies

In [13]:
def remove_rated_movies(movie_ids: pd.Series, rated_movies: pd.Series) -> pd.Series:
    # https://stackoverflow.com/questions/69774160/how-to-delete-values-from-one-pandas-series-that-are-common-to-another
    return movie_ids[~np.isin(movie_ids, rated_movies)]

In [14]:
# TODO (optimizacion): para sacar los ids rateados mas rapido, se puede decorar user ratings
# con el indice de la pelicula en movies para así hacer drop en remove_rated_movies.

## Evaluación

Para usar MARK y MAPK, tenemos que pasar dos listas de la misma longitud que representan los ratings reales (*actual*) y
los ratings predichos (*predicted*).

Para separar el dataset en train y test, vamos a sacar 10 películas de cada usuario. Luego vamos a predecir 10 para cada uno y
aplicar MARK y MAPK para evaluar cada modelo.

Previamente vamos a filtrar el dataset de todos los usuarios que no tengan al menos 15 películas, para tener suficientes para entrenar.

In [15]:
# 1. Sacar películas para las que no tenemos metadata
# hay algunas películas como "253768" que están en ratings pero no en movies.
rated_movies = user_ratings_small["tmdbId"]
rated_movies_with_metadata = rated_movies[rated_movies.isin(movies["id"])]
metadata_filtered_user_ratings = user_ratings_small[user_ratings_small["tmdbId"].isin(rated_movies_with_metadata)]

print(f"total: {len(user_ratings_small)}, after filter: {len(metadata_filtered_user_ratings)}")

total: 26010786, after filter: 25981582


In [16]:
# 2. Sacar los que tienen menos de 15
user_rating_count = metadata_filtered_user_ratings.groupby(["userId"]).count()
users_to_remove = user_rating_count[user_rating_count["movieId"] < 15].reset_index()["userId"]
filtered_user_ratings = metadata_filtered_user_ratings[~metadata_filtered_user_ratings["userId"].isin(users_to_remove)]

print(f"total: {len(metadata_filtered_user_ratings)}, after filter: {len(filtered_user_ratings)}")
print(f"(diff = {len(metadata_filtered_user_ratings) - len(filtered_user_ratings)})")

total: 25981582, after filter: 25556154
(diff = 425428)


In [17]:
def split_into_chunks(elems: list, chunks: int):
    chunk_size = len(elems)//chunks
    rem = len(elems)%chunks
    chunks_split = [ elems[chunk_size*i:chunk_size*(i+1)] for i in range(0, chunks)]

    # Agregamos el resto al último
    chunks_split[chunks-1].extend(elems[len(elems) - rem:])

    return chunks_split

In [37]:
%%time
# 3. Sacar 10 de cada uno para test
NUM_PROCS = 6

from multiprocess import Process, Manager
from typing import List
# https://stackoverflow.com/questions/10415028/how-can-i-recover-the-return-value-of-a-function-passed-to-multiprocessing-proce

def sample_test_ratings(procnum: int, return_dict, users: List[int]):
    print(f"[{procnum}] start")
    test = pd.DataFrame(columns=filtered_user_ratings.columns)

    for user_id in tqdm(users, position=procnum, desc=f" proc #{procnum}"):
        movies_of_user = filtered_user_ratings[filtered_user_ratings["userId"] == user_id].sample(n=10)
        test = pd.concat([test, movies_of_user])

    return_dict[procnum] = test
    print(f"[{procnum}] finish")

user_ids = list(filtered_user_ratings["userId"].unique())
users_split = split_into_chunks(user_ids, NUM_PROCS)

procs = []
manager = Manager()
return_dict = manager.dict()
for i, chunk in enumerate(users_split):
    p = Process(target=sample_test_ratings, args=(i, return_dict, chunk))
    p.start()
    procs.append(p)

for p in procs:
    p.join()

test = pd.DataFrame(columns=filtered_user_ratings.columns)
for return_value in return_dict.values():
    test = pd.concat([test, return_value])
    
train = filtered_user_ratings.drop(test.index)

print("Finished!")

[0] start
[1] start[2] start

[3] start[4] start

[5] start




[2] finish



/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


[0] finish


/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


...



[5] finish


/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


[3] finish
[4] finish


/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


Finished!
CPU times: user 2min 23s, sys: 1min 6s, total: 3min 29s
Wall time: 37min 35s


In [38]:
len(train) + len(test)
# 25556154

25556154

In [39]:
train.to_csv("dfs/collab-train.csv")
test.to_csv("dfs/collab-test.csv")

In [18]:
train = pd.read_csv("dfs/collab-train.csv", index_col=0)
test = pd.read_csv("dfs/collab-test.csv", index_col=0)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:
train = pd.read_csv("dfs/collab-train.csv", index_col=0, dtype={"tmdbId": int})
test = pd.read_csv("dfs/collab-test.csv", index_col=0, dtype={"tmdbId": int})

In [23]:
# 3. Entrenamos el modelo con la data de train
# user_ratings_small_sup = user_ratings_small[['userId', 'tmdbId', 'rating']].dropna()
reader = surprise.Reader(rating_scale=(1, 5))
data = surprise.Dataset.load_from_df(train[['userId', 'tmdbId', 'rating']], reader)

In [24]:
%%time
svd = surprise.SVD()
trainset = data.build_full_trainset()
svd.fit(trainset)
print("Fit done")

Fit done
CPU times: user 19min 9s, sys: 1min 6s, total: 20min 16s
Wall time: 21min 27s


In [25]:
import average_precision
import recmetrics.metrics

In [29]:
%%time
from multiprocess import Process, Manager, Lock
from typing import List
# https://stackoverflow.com/questions/10415028/how-can-i-recover-the-return-value-of-a-function-passed-to-multiprocessing-proce

def predict(i: int, return_dict, users: List[int]):
    actual = []
    predicted = []
    print(f"[{i}]: start")
    for user_id in tqdm(users, position=i, desc=f" proc #{i}"):
        predicted_movies = predict_ratings_for_movies(svd, user_id, train)["tmdbId"]
        actual_movies = test[test["userId"] == user_id]["tmdbId"]
        
        predicted.append(list(predicted_movies))
        actual.append(list(actual_movies))
        
    return_dict[i] = {
        "pred": predicted,
        "actual": actual,
    }

    print(f"[{i}]: finish")

users_split = split_into_chunks(list(train["userId"].unique()), 6)

procs = []
manager = Manager()
return_dict = manager.dict()
for i, chunk in enumerate(users_split):
    p = Process(target=predict, args=(i, return_dict, chunk))
    p.start()
    procs.append(p)

for p in procs:
    p.join()

predicted = []
actual = []

for return_value in return_dict.values():slack

    predicted.extend(return_value["pred"])
    actual.extend(return_value["actual"])
print("Finished!")

[0]: start[1]: start

[2]: start
[3]: start
[4]: start
[5]: start


KeyboardInterrupt: 

Exception ignored in: Exception ignored in: <function tqdm.__del__ at 0x7fbd23560430><function tqdm.__del__ at 0x7fbd23560430>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/mpanichelli/.local/lib/python3.9/site-packages/tqdm/std.py", line 1127, in __del__
  File "/Users/mpanichelli/.local/lib/python3.9/site-packages/tqdm/std.py", line 1127, in __del__
        def __del__(self):def __del__(self):

KeyboardInterruptKeyboardInterrupt: : 

Exception ignored in: <function tqdm.__del__ at 0x7fbd23560430>
Traceback (most recent call last):
  File "/Users/mpanichelli/.local/lib/python3.9/site-packages/tqdm/std.py", line 1127, in __del__
    def __del__(self):
KeyboardInterrupt: 
Exception ignored in: <function tqdm.__del__ at 0x7fbd23560430>
Traceback (most recent call last):
  File "/Users/mpanichelli/.local/lib/python3.9/site-packages/tqdm/std.py", line 1127, in __del__
    def __del__(self):
KeyboardInterrupt: 
Exception ignored in: <function tqdm.__d

In [28]:
predicted = []
actual = []

users = list(train["userId"].unique())
for user_id in tqdm(users):
    predicted_movies = predict_ratings_for_movies(svd, user_id, train)["tmdbId"]
    actual_movies = test[test["userId"] == user_id]["tmdbId"]

    predicted.append(list(predicted_movies))
    actual.append(list(actual_movies))

/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


KeyboardInterrupt: 

/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/opt/homebrew/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


In [ ]:
mark = recmetrics.metrics.mark(actual, predicted, k=10)
mapk = average_precision.mapk(actual, predicted, k=10)
mark, mapk
# (0.0027199076980248984, 0.0027199076980248984)

In [ ]:
# test is in
df = pd.DataFrame({
    'ids': [1, 2, 3, 4],
    'values': ["a", "b", "c", "d"]
})

to_remove = pd.Series([1, 2])
df[~df["ids"].isin(to_remove)]

In [ ]:
# TODO: Tal vez da 0,0 porque los tipos de ids de actual y predicted son int y string.